# Just a nice stacking ensemble model...

In [1]:
import os,sys,time,random,math,time
import tarfile, zipfile

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.cluster import KMeans,MeanShift

from sklearn.base import clone as skclone
from sklearn.externals import joblib

from sklearn.preprocessing import MinMaxScaler
import itertools

import matplotlib.pyplot as plt
from IPython.display import display, Image

import xgboost as xgb

use_xgb=True #disable for speed

from subprocess import check_output
datadir="./input/"
cachedir="./cache/"
outdir='./output/'

print(check_output(["ls", datadir]).decode("utf8"))
print(check_output(["ls", cachedir]).decode("utf8"))
print(check_output(["ls", outdir]).decode("utf8"))


%matplotlib inline  


test.csv
test.csv.zip
train.csv
train.csv.zip

clusters_cat.npy
clusters_cat.npy_01.npy
clusters_cat.npy_02.npy
clusters_cont.npy
clusters_cont.npy_01.npy
clusters_cont.npy_02.npy
clusters.npy
clusters.npy_01.npy
clusters.npy_02.npy
grid_L2_KNN.pkl
grid_L2_Lin.pkl
grid_regr0.pkl
grid_regr1.pkl
grid_regr2.pkl
grid_regr3.pkl
MAE_tracking.npy
oldmodels
x_layer2.npy
x_layer2.npy_01.npy
x_layer2_w_clusters.npy
x_layer2_w_clusters.npy_01.npy



In [2]:
def loadData(datadir,filename):
    # Load the wholesale customers dataset
    #data = pd.read_csv(filename)
    data = ''
    print ("loading: "+datadir+filename)
    try:
        if zipfile.is_zipfile(datadir+filename):
            z = zipfile.ZipFile(datadir+filename)
            filename = z.open(filename[:-4])
        else:
            filename=datadir+filename
        data = pd.read_csv(filename, parse_dates=True)  
        print ("Dataset has {} samples with {} features each.".format(*data.shape))
    except Exception as e:
        print ("Dataset could not be loaded. Is the dataset missing?")
        print(e)
    return data

def writeData(data,filename):
    # Load the wholesale customers dataset
    try:
        data.to_csv(filename, index=False)
    except Exception as e:
        print ("Dataset could not be written.")
        print(e)
    verify=[]
    try:
        with open(filename, 'r') as f:
            for line in f:
                verify.append(line)
        f.closed
        return verify[:5]
    except IOError:
        sys.std
        

In [3]:

def LabelEncoder(data):
    # lifted in parts from:
    #https://www.kaggle.com/mmueller/allstate-claims-severity/yet-another-xgb-starter/code
    features = data.columns
    cats = [feat for feat in features if 'cat' in feat]
    for feat in cats:
        data[feat] = pd.factorize(data[feat], sort=True)[0]
    return data

In [4]:
def kmeansPlusmeanshift(data):
    start_time = time.time()
    startingClusterSize=int(len(data)*.075)
    print "kmeans.... for {} clusters".format(startingClusterSize)
    k_means =KMeans(n_clusters=startingClusterSize,n_jobs=10)
    k_means.fit(data.sample(frac=0.35).values)
    clusters=k_means.cluster_centers_
    print("kmeans round 1 time:{}s".format(round((time.time()-start_time), 3) ))
    print clusters[:15]
    
    start_time = time.time()
    #use the cluster centers of the guessed clusters to get an estimate of actual numbers of clusters. doing this for speed increase!
    print "\nmeanshift..."
    meanshift=MeanShift(n_jobs=10)
    meanshift.fit(clusters)
    newcenters=meanshift.cluster_centers_
    print("meanshift time:{}s".format(round((time.time()-start_time), 3) ))
    print newcenters[:15], "\nnum of clusters from meanshift:",len(newcenters)
    
    start_time = time.time()
    # use the new clusters number to predict each locations cluster
    print "\nredo kmeans with new cluster number from meanshift +1 to account for sampling..."
    k_means =KMeans(n_clusters=len(newcenters)+1,n_jobs=10)
    final_clusters=k_means.fit_predict(data.values)
    print("kmeans round 2 time:{}s".format(round((time.time()-start_time), 3) ))
    return final_clusters

In [5]:
def kmeansPlusmeanshift_quick(data):  # used the one above to get the # of clusters, using this for speed
    start_time = time.time()
    # use the new clusters number to predict each locations cluster
    print "\nredo kmeans with new cluster number from meanshift +1 to account for sampling..."
    k_means =KMeans(n_clusters=80,n_jobs=12)
    final_clusters=k_means.fit_predict(data.values)
    print("kmeans round 2 time:{}s".format(round((time.time()-start_time), 3) ))
    return final_clusters

In [30]:
data = loadData(datadir,'train.csv.zip')
display(data.info())
display(data.head(5))

test_data= loadData(datadir,'test.csv.zip') 
display(test_data.info())
display(test_data.head(5))

loading: ./input/train.csv.zip
Dataset has 188318 samples with 132 features each.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, id to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 189.7+ MB


None

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


loading: ./input/test.csv.zip
Dataset has 125546 samples with 131 features each.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125546 entries, 0 to 125545
Columns: 131 entries, id to cont14
dtypes: float64(14), int64(1), object(116)
memory usage: 125.5+ MB


None

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


###  Pre Proccessing

In [31]:
# combine the two frames so we can encode the labels!
test_data['loss']=0

lengthofData=len(data)
lengthoftest_data=len(test_data)

print("data:",lengthofData)
print("test:",lengthoftest_data)

combineddata=pd.concat([data,test_data])
lengthofcombined=len(combineddata)
print("combined:",lengthofcombined)

orig_features = combineddata.columns

('data:', 188318)
('test:', 125546)
('combined:', 313864)


In [32]:
# the categorical data that we need in a number format
combineddata=LabelEncoder(combineddata)
print("label encoded")

label encoded


In [33]:
hold_columns=combineddata[['loss','id']] #don't want to mess with these

combineddata.drop(['loss','id'],1,inplace=True) 
columns=combineddata.columns #need these to recreate the DF

#scale the columns we see
scaler= MinMaxScaler()   
values = scaler.fit_transform(combineddata.values)
combineddata= pd.DataFrame(values, columns=columns)

#put these back on for the moment!
combineddata['loss']=hold_columns['loss'].tolist()
combineddata['id']=hold_columns['id'].tolist()
del hold_columns
del values
del scaler
print "Data scaled"

Data scaled


In [34]:
# taken from Ali's script (https://www.kaggle.com/aliajouz/allstate-claims-severity/singel-model-lb-1117)
combineddata["cont1"] = np.sqrt(combineddata["cont1"])
combineddata["cont4"] = np.sqrt(combineddata["cont4"])
combineddata["cont5"] = np.sqrt(combineddata["cont5"])
combineddata["cont8"] = np.sqrt(combineddata["cont8"])
combineddata["cont10"] = np.sqrt(combineddata["cont10"])
combineddata["cont11"] = np.sqrt(combineddata["cont11"])
combineddata["cont12"] = np.sqrt(combineddata["cont12"])

combineddata["cont6"] = np.log(combineddata["cont6"] + 0000.1)
combineddata["cont7"] = np.log(combineddata["cont7"] + 0000.1)
combineddata["cont9"] = np.log(combineddata["cont9"] + 0000.1)
combineddata["cont13"] = np.log(combineddata["cont13"] + 0000.1)
combineddata["cont14"] = (np.maximum(combineddata["cont14"] - 0.179722, 0) / 0.665122) ** 0.25


In [35]:
#predict the cluster for each row
filename='clusters.npy'
if os.path.isfile(cachedir+filename):
    print("File found, using it")
    combineddata['clusters']=joblib.load(cachedir+filename)
else:
    print("no files, running clusters...")
    combineddata['clusters']=kmeansPlusmeanshift_quick(combineddata.drop(['id','loss'],1))
    joblib.dump(combineddata['clusters'],cachedir+filename)
print("clusters loaded and attached")
print(combineddata.head(3)['clusters'])

File found, using it
clusters loaded and attached
0    57
1    29
2    23
Name: clusters, dtype: int32


In [36]:
#predict the cluster for cont for each row
filename='clusters_cont.npy'
cats = [feat for feat in data.columns if 'cat' in feat]
if os.path.isfile(cachedir+filename):
    print("File found, using it")
    combineddata['clusters_cont']=joblib.load(cachedir+filename)
else:
    print("no files, running clusters...")
    combineddata['clusters_cont']=kmeansPlusmeanshift_quick(combineddata.drop(['id','loss'],1).drop(cats,1))
    joblib.dump(combineddata['clusters_cont'],cachedir+filename)
print("clusters_cont loaded and attached")
print(combineddata.head(3)['clusters_cont'])

File found, using it
clusters_cont loaded and attached
0    14
1    44
2    53
Name: clusters_cont, dtype: int32


In [37]:
#predict the cluster for cont for each row
filename='clusters_cat.npy'
conts = [feat for feat in data.columns if 'cont' in feat]
if os.path.isfile(cachedir+filename):
    print("File found, using it")
    combineddata['clusters_cat']=joblib.load(cachedir+filename)
else:
    print("no files, running clusters...")
    combineddata['clusters_cat']=kmeansPlusmeanshift_quick(combineddata.drop(['id','loss'],1).drop(conts,1))
    joblib.dump(combineddata['clusters_cat'],cachedir+filename)
print("clusters_cat loaded and attached")
print(combineddata.head(3)['clusters_cat'])

File found, using it
clusters_cat loaded and attached
0    31
1     9
2    29
Name: clusters_cat, dtype: int32


In [38]:
# scale the clusters

#scale the columns we see
cluster_scaler= MinMaxScaler()   
combineddata['clusters']= cluster_scaler.fit_transform(combineddata['clusters'].values)
combineddata['clusters_cont']= cluster_scaler.fit_transform(combineddata['clusters_cont'].values)
combineddata['clusters_cat']= cluster_scaler.fit_transform(combineddata['clusters_cat'].values)


/usr/local/lib/python2.7/dist-packages/sklearn/utils/validation.py:420: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:324: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/preprocessing/data.py:359: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
/usr/local/lib/python2.7/dist-packages/sklearn/utils/vali

In [39]:
display(data.info())
display(data.head(3))
# time to split the data back apart!
data=combineddata.iloc[:lengthofData].copy()
test_data=combineddata.iloc[lengthofData:].copy()


print("Pre-Processing done")
display(data.info())
display(data.head(3))
print("data:",len(x))
print("labels:",len(y))
print("test:",len(x_test_data))


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, id to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 189.7+ MB


None

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09


Pre-Processing done
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 135 entries, cat1 to clusters_cat
dtypes: float64(134), int64(1)
memory usage: 194.0 MB


None

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont10,cont11,cont12,cont13,cont14,loss,id,clusters,clusters_cont,clusters_cat
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.916140,0.744792,0.761787,-0.070392,0.984628,2213.18,1,0.721519,0.177215,0.392405
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.664384,0.560798,0.585682,-0.330645,0.343682,1283.60,2,0.367089,0.556962,0.113924
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.571049,0.599347,0.591963,-1.211326,1.018094,3005.09,5,0.291139,0.670886,0.367089


('data:', 188318)
('labels:', 188318)
('test:', 125546)


In [54]:
# Data with all features kept
display (data.head(3))
writeData(data,datadir+'data_all_features.csv')
writeData(test_data,datadir+'test_data_all_features.csv')

#Data with no created features
#print list(orig_features)
data_orig_only= data[list(orig_features)].copy()
data_orig_only['loss']=data['loss']
data_orig_only['id']=data['id']
test_data_orig_only= test_data[list(orig_features)].copy()
test_data_orig_only['id']=test_data['id']
display (data_orig_only.head(3))
writeData(data_orig_only,datadir+'data_orig_only.csv')
writeData(test_data_orig_only,datadir+'test_data_orig_only.csv')

# orig Data, cats only
cats = [feat for feat in orig_features if 'cat' in feat]
data_cats=data[list(cats)].copy()
data_cats['loss']=data['loss']
data_cats['id']=data['id']
test_data_cats= test_data[list(orig_features)].copy()
test_data_cats['id']=test_data['id']
display (data_cats.head(3))
writeData(data_cats,datadir+'data_cats.csv')
writeData(test_data_cats,datadir+'test_data_cats.csv')

# orig Data, conts only
conts = [feat for feat in orig_features if 'cont' in feat]
data_conts=data[list(conts)].copy()
data_conts['loss']=data['loss']
data_conts['id']=data['id']
test_data_conts=test_data[list(conts)].copy()
test_data_conts['id']=test_data['id']
display (data_conts.head(3))
writeData(data_conts,datadir+'data_conts.csv')
writeData(test_data_conts,datadir+'test_data_conts.csv')

# Data, new only
new_features=data.columns.difference(orig_features)
data_new=data[list(new_features)].copy()
data_new['loss']=data['loss']
data_new['id']=data['id']
test_data_data_new=test_data[list(new_features)].copy()
test_data_data_new['id']=test_data['id']
display (data_new.head(3))
writeData(data_new,datadir+'data_new.csv')
writeData(test_data_data_new,datadir+'test_data_data_new.csv')



,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont10,cont11,cont12,cont13,cont14,loss,id,clusters,clusters_cont,clusters_cat
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.916140,0.744792,0.761787,-0.070392,0.984628,2213.18,1,0.721519,0.177215,0.392405
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.664384,0.560798,0.585682,-0.330645,0.343682,1283.60,2,0.367089,0.556962,0.113924
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.571049,0.599347,0.591963,-1.211326,1.018094,3005.09,5,0.291139,0.670886,0.367089


,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,-0.202349,-0.953493,0.296826,-0.255633,0.916140,0.744792,0.761787,-0.070392,0.984628,2213.18
1,2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.629320,-0.704206,0.698552,-0.792214,0.664384,0.560798,0.585682,-0.330645,0.343682,1283.60
2,5,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,-0.964086,-1.009448,0.220662,-1.016372,0.571049,0.599347,0.591963,-1.211326,1.018094,3005.09


,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cat109,cat110,cat111,cat112,cat113,cat114,cat115,cat116,loss,id
0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.547619,0.210526,0.125,0.38,0.919355,0.0,0.636364,0.827586,2213.18,1
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,0.404762,0.503759,0.000,0.44,0.629032,0.0,0.636364,0.270115,1283.60,2
2,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,...,0.023810,0.654135,0.000,0.56,0.096774,0.0,0.363636,0.479885,3005.09,5


,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss,id
0,0.858705,0.284121,0.196416,0.886802,0.202886,-0.202349,-0.953493,0.296826,-0.255633,0.916140,0.744792,0.761787,-0.070392,0.984628,2213.18,1
1,0.579262,0.854225,0.626632,0.749105,0.927757,-0.629320,-0.704206,0.698552,-0.792214,0.664384,0.560798,0.585682,-0.330645,0.343682,1283.60,2
2,0.515580,0.414588,0.511420,0.277513,0.406217,-0.964086,-1.009448,0.220662,-1.016372,0.571049,0.599347,0.591963,-1.211326,1.018094,3005.09,5


,clusters,clusters_cat,clusters_cont,loss,id
0,0.721519,0.392405,0.177215,2213.18,1
1,0.367089,0.113924,0.556962,1283.60,2
2,0.291139,0.367089,0.670886,3005.09,5


['clusters,clusters_cat,clusters_cont,id\n',
 '0.367088607595,0.113924050633,0.556962025316,4\n',
 '0.443037974684,0.835443037975,0.53164556962,6\n',
 '0.354430379747,0.417721518987,0.291139240506,9\n',
 '0.215189873418,0.240506329114,0.303797468354,12\n']

In [ ]:
# in the next layer add these:

# we don't want the ID columns in X
x=data.drop(['id','loss'],1).values
# loss is our label
#y=data['loss'].values
shift=200
y = np.log(data['loss']+shift).ravel()

x_test_data=test_data.drop(['loss','id'],1) .values# didn't have the loss column before, make it go away! don't need ID!




# EOF 